In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import numpy.fft as fft
import numpy.polynomial.chebyshev as nc
import argparse as ap
import os, sys, time, ld
import mpmath as mm

mm.mp.dps = 30
import astropy.io.fits as ps
from multiprocessing import Pool, Manager
import gc
import psr_read as pr
import time_eph as te
import psr_model as pm
import subprocess as sp

<IPython.core.display.Javascript object>

In [3]:
filelist = [
    "/home/juianhsu/2019a-109-P/J0621+1002/20190702/J0621+1002_tracking_0001.fits",
    "/home/juianhsu/2019a-109-P/J0621+1002/20190702/J0621+1002_tracking_0002.fits",
    "/home/juianhsu/2019a-109-P/J0621+1002/20190702/J0621+1002_tracking_0003.fits",
]
filenum = len(filelist)
file_t0 = []
file_time = []
file_len = []

<IPython.core.display.Javascript object>

In [4]:
(
    telename,
    pol_type,
    npol,
    nchan,
    freq,
    bandwidth,
    tsamp,
    nsblk,
    bw_sign,
    stt_imjd,
    stt_smjd,
    stt_offs,
    nsub,
    offs_sub,
) = ("", "", 0, 0, 0, 0.0, 0.0, 0, True, 0, 0, 0.0, 0, 0.0)

<IPython.core.display.Javascript object>

# file check

In [5]:
def file_check(fname, notfirst=True, filetype="data"):
    if not os.path.isfile(fname):
        print("Fits " + filetype + " name is invalid.")
    try:
        f = ps.open(fname, mmap=True)  # f=ps.open(filelist[i],mmap=True)
    except:
        print("Fits " + filetype + " is invalid.")
    head = f["PRIMARY"].header
    subint = f["SUBINT"]
    subint_header = subint.header
    subint_data = subint.data[0]
    global telename, pol_type, npol, nchan, freq, bandwidth, tsamp, nsblk, bw_sign, stt_imjd, stt_smjd, stt_offs, nsub
    if not notfirst:
        telename = head["TELESCOP"]
        npol = subint_header["NPOL"]
        nchan = head["OBSNCHAN"]
        freq = head["OBSFREQ"]
        bandwidth = subint_header["NCHAN"] * subint_header["CHAN_BW"]
        bw_sign = bandwidth > 0
        bandwidth = np.abs(bandwidth)
        tsamp = subint_header["TBIN"]
        nsblk = subint_header["NSBLK"]
        pol_type = subint_header["POL_TYPE"]
    else:
        if telename != head["TELESCOP"]:
            file_error("telescope name", filetype)
        if pol_type != subint_header["POL_TYPE"]:
            file_error("polarisation type", filetype)
        if npol != subint_header["NPOL"]:
            file_error("number of polorisations", filetype)
        if nchan != head["OBSNCHAN"]:
            file_error("number of channels", filetype)
        if freq != head["OBSFREQ"]:
            file_error("central frequency", filetype)
        if bandwidth != np.abs(subint_header["NCHAN"] * subint_header["CHAN_BW"]):
            file_error("bandwidth", filetype)
        if tsamp != subint_header["TBIN"]:
            file_error("sampling time", filetype)
        #
    stt_imjd = head["STT_IMJD"]
    stt_smjd = head["STT_SMJD"]
    stt_offs = head["STT_OFFS"]
    nsub = subint_header["NAXIS2"]
    offs_sub = subint_data["OFFS_SUB"]
    del subint_data
    f.close()

<IPython.core.display.Javascript object>

In [6]:
for i in np.arange(filenum):
    file_check(filelist[i], notfirst=i)
    subint_t0 = (
        offs_sub - tsamp * nsblk / 2.0 + stt_smjd + stt_offs
    ) / 86400.0 + stt_imjd
    file_time.append([offs_sub - tsamp * nsblk / 2.0, stt_smjd, stt_offs, stt_imjd])
    file_len.append(nsub)
    file_t0.append(subint_t0)

<IPython.core.display.Javascript object>

In [7]:
filenum, file_t0, file_time, file_len

(3,
 [58666.14930526428, 58666.14937982969, 58666.14945439509],
 [[-0.025165824, 12900, 0.0, 58666],
  [-0.025165824, 12906, 0.442450944, 58666],
  [-0.025165824, 12912, 0.884901888, 58666]],
 [128, 128, 128])

<IPython.core.display.Javascript object>

In [8]:
file_len, file_t0, filelist, file_time = (
    np.array(file_len),
    np.array(file_t0),
    np.array(filelist),
    np.array(file_time),
)
sorts = np.argsort(file_t0)
file_len, file_t0, filelist, file_time = (
    file_len[sorts],
    np.sort(file_t0),
    filelist[sorts],
    file_time[sorts],
)
if len(file_len) > 1:
    if np.max(
        np.abs((file_len * nsblk * tsamp / 86400.0 + file_t0)[:-1] - file_t0[1:])
    ) > (tsamp / 86400.0):
        parser.error("Data files are not continuous.")

<IPython.core.display.Javascript object>

# cal

In [9]:
noise_mark = "ld"
noise = ld.ld("/home/juianhsu/2019a-109-P/J0621+1002/20190702/OFF_tracking_1.ld")
noise_info = noise.read_info()
if noise_info["mode"] != "cal":
    print("LD file is not caliration file.")
elif telename != noise_info["telename"]:
    print("LD calibration file has different telescope name.")
elif nchan != int(noise_info["nchan"]):
    print("LD calibration file has different channel number.")

<IPython.core.display.Javascript object>

# freqrange

In [10]:
channel_width = bandwidth * 1.0 / nchan
chanstart, chanend = 0, nchan
nchan_new = chanend - chanstart

<IPython.core.display.Javascript object>

In [11]:
nchan_new

4096

<IPython.core.display.Javascript object>

In [12]:
nbin = file_len.sum() * nsblk
stt_time = file_t0[0]
freq_start, freq_end = (
    np.array([chanstart, chanend]) - 0.5 * nchan
) * channel_width + freq
info = {
    "nbin_origin": nbin,
    "telename": telename,
    "freq_start": freq_start,
    "freq_end": freq_end,
    "nchan": chanend - chanstart,
    "tsamp_origin": tsamp,
    "stt_time_origin": stt_time,
    "stt_time": stt_time,
    "npol": npol,
}

<IPython.core.display.Javascript object>

In [13]:
info

{'nbin_origin': 393216,
 'telename': 'FAST',
 'freq_start': 1000.0,
 'freq_end': 1500.0,
 'nchan': 4096,
 'tsamp_origin': 4.9152e-05,
 'stt_time_origin': 58666.14930526428,
 'stt_time': 58666.14930526428,
 'npol': 4}

<IPython.core.display.Javascript object>

# psr name

In [14]:
psr_name = "J0621+1002"
psr = pr.psr(psr_name)
psr_par = sp.getoutput("psrcat -e " + psr_name).split("\n")
if len(psr_par) < 3:
    print("A valid pulsar name is required.")
for i in range(len(psr_par)):
    psr_par[i] = psr_par[i] + "\n"

<IPython.core.display.Javascript object>

In [15]:
info["psr_par"] = psr_par
pepoch = False
for line in psr_par:
    elements = line.split()
    if elements[0] == "PSRJ":
        psr_name = elements[1].strip("\n")
        info["psr_name"] = psr_name
    elif elements[0] == "F0":
        period = 1.0 / np.float64(elements[1])
    elif elements[0] == "DM":
        dm = np.float64(elements[1])
    elif elements[0] == "PEPOCH":
        pepoch = True

<IPython.core.display.Javascript object>

# sunint

In [16]:
info["mode"] = "single"

<IPython.core.display.Javascript object>

In [17]:
info["dm"] = dm

<IPython.core.display.Javascript object>

In [18]:
dm

36.47

<IPython.core.display.Javascript object>

# zap

In [19]:
zchan = []

<IPython.core.display.Javascript object>

# output

In [20]:
name = "tracking_0001_0003"
if len(name) > 3:
    if name[-3:] == ".ld":
        name = name[:-3]

<IPython.core.display.Javascript object>

In [21]:
noise_a12, noise_a22, noise_cos, noise_sin = noise.read_period(0)[
    chanstart:chanend, 0
].T

<IPython.core.display.Javascript object>

# cal

In [22]:
info["cal"] = list(
    map(str, np.array([noise_a12, noise_a22, noise_cos, noise_sin]).T.reshape(-1))
)
noise_a12 = np.where(noise_a12 > 0, 1.0 / noise_a12, 0)
noise_a22 = np.where(noise_a22 > 0, 1.0 / noise_a22, 0)
noise_a1a2 = noise_a12 * noise_a22
noise_a1a2 = np.sqrt(noise_a1a2)
noise_cos = noise_cos * noise_a1a2
noise_sin = noise_sin * noise_a1a2
noise_a1p2 = (noise_a12 + noise_a22) / 2.0
noise_a1m2 = (noise_a12 - noise_a22) / 2.0

<IPython.core.display.Javascript object>

In [23]:
nbin_old = nbin
freq0 = freq_start
freq1 = freq_end
nbin0 = nbin
#
roach_delay = 8.192e-6 * 3
gps_delay = 1e-5
transline_delay = 2e-5
light_delay = (300.0 + 141.96) / 3.0e8
delay = transline_delay + light_delay - gps_delay - roach_delay
#
stt_time = info["stt_time"]
end_time = stt_time + tsamp * nbin0 / 86400.0 + 60.0 / 86400
stt_time -= 60.0 / 86400
stt_time = str(int(stt_time)) + str(stt_time % 1)[1:]
end_time = str(int(end_time)) + str(end_time % 1)[1:]
time0 = file_time[0]

<IPython.core.display.Javascript object>

# period

In [24]:
ncoeff = 12
chebx_test = nc.chebpts1(ncoeff)
second_test = (chebx_test + 1) / 2 * nbin0 * tsamp + file_time[0][:-1].sum() - delay
time_test = te.time(file_time[0][-1] * np.ones(ncoeff), second_test, scale="local")
times_test = te.times(time_test)
timing_test_end = pm.psr_timing(psr, times_test, freq_end)
timing_test_start = pm.psr_timing(psr, times_test, freq_start)
phase_start = timing_test_end.phase.date[0] + 1
phase_end = timing_test_start.phase.date[-1]
phase = timing_test_end.phase.date - phase_start + timing_test_end.phase.second
nperiod = phase_end - phase_start
period = (
    (time_test.date[-1] - time_test.date[0]) * time_test.unit
    + time_test.second[-1]
    - time_test.second[0]
) / (
    timing_test_end.phase.date[-1]
    - timing_test_end.phase.date[0]
    + timing_test_end.phase.second[-1]
    - timing_test_end.phase.second[0]
)
cheb_end = nc.chebfit(chebx_test, timing_test_end.phase.date - phase_start, ncoeff - 1)
roots = nc.chebroots(cheb_end)
roots = np.real(roots[np.isreal(roots)])
root = roots[np.argmin(np.abs(roots + 1))]
stt_time_test = te.time(
    file_time[0][-1],
    (root + 1) / 2 * nbin0 * tsamp + file_time[0][:-1].sum() - delay,
    scale="local",
)
stt_sec = stt_time_test.second
stt_date = stt_time_test.date
info["phase0"] = phase_start
ncoeff_freq = 10
phase_tmp = np.zeros([ncoeff_freq, ncoeff])
disp_tmp = np.zeros(ncoeff_freq)
cheby = nc.chebpts1(ncoeff_freq)
freqy = (cheby + 1) / 2 * bandwidth + freq_start
for i in np.arange(ncoeff_freq):
    timing_test = pm.psr_timing(psr, times_test, freqy[i])
    disp_tmp[i] = (
        (timing_test.tdis1 + timing_test.tdis2) / timing_test.period_now
    ).mean()
    phase_tmp[i] = (
        timing_test.phase.date - phase_start + timing_test.phase.second + disp_tmp[i]
    )
coeff_freq = np.polyfit(1 / freqy, disp_tmp, 4)
coeff = nc.chebfit(chebx_test, nc.chebfit(cheby, phase_tmp, 1).T, ncoeff - 1)
info["predictor"] = list(map(list, coeff))
info["predictor_freq"] = list(coeff_freq)

<IPython.core.display.Javascript object>

In [25]:
nperiod

660

<IPython.core.display.Javascript object>

In [26]:
phase_start + nperiod

10977904487

<IPython.core.display.Javascript object>

In [27]:
phase_end

10977904487

<IPython.core.display.Javascript object>

In [28]:
info["stt_sec"] = stt_sec[0]
info["stt_date"] = stt_date[0]
info["stt_time"] = stt_date[0] + stt_sec[0] / 86400.0
#
info["nperiod"] = nperiod
info["period"] = period
#
nbin_max = (nbin0 - 1) / (np.max(phase) - np.min(phase))

<IPython.core.display.Javascript object>

# nbins

In [29]:
nbin = 2 ** np.int16(np.log2(nbin_max))
temp_multi = 1

<IPython.core.display.Javascript object>

In [30]:
info["nbin"] = nbin
info["length"] = period * nperiod
#
totalbin = nperiod * nbin * temp_multi
dphasebin = 1.0 / (nbin * temp_multi)
df = freq_start + np.arange(nchan_new) * channel_width
df0 = (df - freq_start) / (freq_end - freq_start) * 2 - 1

<IPython.core.display.Javascript object>

In [31]:
info["nbin"]

512

<IPython.core.display.Javascript object>

# ld file

In [32]:
d = ld.ld(name + ".ld")

<IPython.core.display.Javascript object>

In [33]:
info["nsub"] = nperiod
info["file_time"] = time.strftime("%Y-%m-%dT%H:%M:%S", time.gmtime())
d.write_shape([nchan_new, info["nsub"], nbin, npol])

<IPython.core.display.Javascript object>

In [37]:
d.write_info(info)

<IPython.core.display.Javascript object>

In [38]:
! du -h tracking_0001_0003.ld

328K	tracking_0001_0003.ld


<IPython.core.display.Javascript object>

In [73]:
def write_data(ldfile, data, startbin, channum, lock=0):
    a1p2, a1m2, ncos, nsin = (
        noise_a1p2[channum],
        noise_a1m2[channum],
        noise_cos[channum],
        noise_sin[channum],
    )
    i, q, u, v = data
    i, q, u, v = (
        a1p2 * i - a1m2 * q,
        a1p2 * q - a1m2 * i,
        ncos * u + nsin * v,
        -nsin * u + ncos * v,
    )
    data = np.array([i, q, u, v])
    d.__write_chanbins_add__(data.T, startbin, channum)

<IPython.core.display.Javascript object>

In [74]:
def gendata(cums, nsub, data, d, tpsub=0, tpsubn=0, last=False, first=True, lock=0):
    data = np.concatenate(
        (np.zeros([2, npol, nchan]), data, np.zeros([2, npol, nchan])), axis=0
    ).transpose(2, 1, 0)
    data = data[chanstart:chanend]
    dt = np.arange(nsblk * cums - 2, nsblk * cums + nsub + 2) / nbin0 * 2 - 1
    for f in np.arange(nchan_new):
        if f + chanstart in zchan:
            continue
        phase = nc.chebval2d(
            dt, df0[f] * np.ones_like(dt, dtype=np.float64), coeff
        ) - np.polyval(coeff_freq, 1 / df[f])
        newphase = np.arange(
            phase[0] // dphasebin + 1, phase[-1] // dphasebin + 1, dtype=np.int64
        )
        if newphase[-1] < 0 or newphase[0] >= totalbin:
            continue
        newphase = newphase[(newphase >= 0) & (newphase < totalbin)]
        tpdata = np.zeros([npol, len(newphase)])
        for p in np.arange(npol):
            tpdata[p] = np.interp(newphase, phase / dphasebin, data[f, p, :])
        if temp_multi > 1:
            startphase, phaseres0 = divmod(newphase[0], temp_multi)
            phaseres0 = np.int64(phaseres0)
            nphase = np.int64(
                np.ceil((newphase[-1] + 1 - newphase[0] + phaseres0) * 1.0 / temp_multi)
            )
            tpdata = (
                np.concatenate(
                    (
                        np.zeros([npol, phaseres0]),
                        tpdata,
                        np.zeros(
                            [
                                npol,
                                nphase * temp_multi
                                - newphase[-1]
                                - 1
                                + newphase[0]
                                - phaseres0,
                            ]
                        ),
                    ),
                    axis=1,
                )
                .reshape(npol, nphase, temp_multi)
                .sum(2)
            )
        else:
            startphase = newphase[0]
            nphase = newphase.size
        if info["mode"] == "single":
            write_data(d, tpdata, startphase, f, lock=lock)
        else:
            startperiod, phaseres1 = divmod(startphase, nbin)
            phaseres1 = np.int64(phaseres1)
            file_nperiod = np.int64(np.ceil((nphase + phaseres1) * 1.0 / nbin))
            startsub, periodres = divmod(startperiod, sub_nperiod)
            periodres = np.int64(periodres)
            file_nsub = np.int64(
                np.ceil((file_nperiod + periodres) * 1.0 / sub_nperiod)
            )
            if file_nsub > 1 or newphase[-1] == totalbin - 1:
                file_sub_data = np.zeros([npol, file_nsub, nbin])
                if newphase[-1] == totalbin - 1:
                    if newphase[0] == 0:
                        tpdata = tpdata.reshape(npol, -1, nbin)
                        file_sub_data[:, :-1] = (
                            tpdata[:, :(-sub_nperiod_last)]
                            .reshape(npol, file_nsub - 1, sub_nperiod, nbin)
                            .sum(2)
                        )
                        file_sub_data[:, -1] = tpdata[:, (-sub_nperiod_last):].sum(1)
                    elif file_nsub > 1:
                        tpsub[f, :, phaseres1:] += tpdata[:, : (nbin - phaseres1)]
                        tpdata = tpdata[:, (nbin - phaseres1) :].reshape(npol, -1, nbin)
                        file_sub_data[:, 1:-1] = (
                            tpdata[
                                :, (sub_nperiod - periodres - 1) : (-sub_nperiod_last)
                            ]
                            .reshape(npol, file_nsub - 2, sub_nperiod_last, nbin)
                            .sum(2)
                        )
                        file_sub_data[:, 0] = tpsub[f] + tpdata[
                            :, : (sub_nperiod - periodres - 1)
                        ].sum(1)
                        file_sub_data[:, -1] = tpdata[:, (-sub_nperiod_last):].sum(1)
                    else:
                        tpsub[f, :, phaseres1:] += tpdata[:, : (nbin - phaseres1)]
                        tpdata = tpdata[:, (nbin - phaseres1) :].reshape(npol, -1, nbin)
                        file_sub_data[:, 0] = tpsub[f] + tpdata.sum(1)
                    write_data(
                        d,
                        file_sub_data.reshape(npol, -1),
                        startsub * nbin,
                        f,
                        lock=lock,
                    )
                else:
                    periodres_left = sub_nperiod - periodres
                    periodres_right = (file_nsub - 1) * sub_nperiod - periodres
                    phaseres_left = periodres_left * nbin - phaseres1
                    phaseres_right = periodres_right * nbin - phaseres1
                    phaseres_left1 = nbin - phaseres1
                    phaseres_right1 = (file_nperiod - 1) * nbin - phaseres1
                    file_sub_data[:, 1:-1] = (
                        tpdata[:, phaseres_left:phaseres_right]
                        .reshape(npol, file_nsub - 2, sub_nperiod, nbin)
                        .sum(2)
                    )
                    file_sub_data[:, 0] = (
                        tpdata[:, phaseres_left1:phaseres_left]
                        .reshape(npol, sub_nperiod - periodres - 1, nbin)
                        .sum(1)
                    )
                    file_sub_data[:, 0, phaseres1:] += tpdata[:, :phaseres_left1]
                    file_sub_data[:, -1] = (
                        tpdata[:, phaseres_right:phaseres_right1]
                        .reshape(npol, file_nperiod - 1 - periodres_right, nbin)
                        .sum(1)
                    )
                    file_sub_data[:, -1, : (nphase - phaseres_right1)] += tpdata[
                        :, phaseres_right1:
                    ]
                    if tpsubn[f] <= startsub:
                        file_sub_data[:, 0] += tpsub[f]
                    else:
                        file_sub_data[:, 1] += tpsub[f]
                    write_data(
                        d,
                        file_sub_data[:, :-1].reshape(npol, -1),
                        startsub * nbin,
                        f,
                        lock=lock,
                    )
                    tpsub[f] = file_sub_data[:, -1]
                    tpsubn[f] = startsub + file_nsub - 1
                    if args.multi and last:
                        write_data(d, tpsub[f], tpsubn[f] * nbin, f, lock=lock)
            else:
                if file_nperiod > 1:
                    phaseres_left = nbin - phaseres1
                    phaseres_right = (file_nperiod - 1) * nbin - phaseres1
                    tpsub[f, :, phaseres1:] += tpdata[:, :phaseres_left]
                    tpsub[f, :, : (nphase - phaseres_right)] += tpdata[
                        :, phaseres_right:
                    ]
                    if file_nperiod > 2:
                        tpsub[f] += (
                            tpdata[:, phaseres_left:phaseres_right]
                            .reshape(npol, file_nperiod - 2, nbin)
                            .sum(1)
                        )
                else:
                    tpsub[f, :, phaseres1 : (phaseres1 + tpdata.shape[1])] += tpdata
    return tpsub, tpsubn

<IPython.core.display.Javascript object>

In [75]:
def dealdata(filelist, n, d, lock=0):
    tpsub = 0
    tpsubn = 0
    f = ps.open(filelist[n], mmap=True)
    fsub = f["SUBINT"].header["naxis2"]
    for i in np.arange(fsub):
        cumsub = np.int64(file_len[:n].sum() + i)
        dtmp = f["SUBINT"].data[i]
        data = np.int16(
            dtmp["DATA"].reshape(nsblk, npol, nchan)
            * dtmp["dat_scl"].reshape(1, npol, nchan)
            + dtmp["dat_offs"].reshape(1, npol, nchan)
        )
        del f["SUBINT"].data
        tpsub, tpsubn = gendata(
            cumsub,
            nsblk,
            data,
            d,
            tpsub,
            tpsubn,
            last=(i == (fsub - 1)),
            first=i == 0,
            lock=lock,
        )
    f.close()
    gc.collect()

<IPython.core.display.Javascript object>

In [76]:
info.keys()

dict_keys(['nbin_origin', 'telename', 'freq_start', 'freq_end', 'nchan', 'tsamp_origin', 'stt_time_origin', 'stt_time', 'npol', 'psr_par', 'psr_name', 'mode', 'dm', 'cal', 'phase0', 'predictor', 'predictor_freq', 'stt_sec', 'stt_date', 'nperiod', 'period', 'nbin', 'length', 'nsub', 'file_time'])

<IPython.core.display.Javascript object>

In [77]:
info["nsub"]

660

<IPython.core.display.Javascript object>

In [78]:
for n in np.arange(filenum):
    if n % 10 == 0 and n > 0:
        d.write_info(info)
        d = ld.ld(name + "_%s.ld" % (n // 10))
        d.write_shape([nchan_new, info["nsub"], nbin, npol])
    dealdata(filelist, n, d)
    if n == filenum - 1:
        d.write_info(info)

<IPython.core.display.Javascript object>

# Read ld

In [43]:
ld_file = ld.ld("/home/juianhsu/2019a-109-P/J0621+1002/20190702/tracking_0001.ld")

In [51]:
print(dir(ld_file))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__read_bin_segment__', '__read_chan0__', '__read_size__', '__reduce__', '__reduce_ex__', '__refresh_size__', '__repr__', '__setattr__', '__size__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '__write_bin_segment__', '__write_chanbins__', '__write_chanbins_add__', '__write_size__', 'chan_scrunch', 'file', 'name', 'period_scrunch', 'read_chan', 'read_data', 'read_info', 'read_para', 'read_period', 'read_shape', 'write_chan', 'write_info', 'write_para', 'write_period', 'write_shape']


In [52]:
ld_file.name

'/home/juianhsu/2019a-109-P/J0621+1002/20190702/tracking_0001.ld'

In [53]:
ld_file.__size__

array([24,  0,  0,  0,  0])

In [54]:
open(ld_file.name,'rb+').seek(0)

0